- [编程常见问题](https://docs.python.org/zh-cn/3/faq/programming.html)

# 1. 列表生成器

## 描述

下面的代码会报错，为什么？


In [1]:
class A(object):
    x = 1
    gen = (x for _ in range(10))
    
print(list(A.gen))    

NameError: name 'x' is not defined

## 答案

这个问题是变量作用域问题，在 `gen=(x for _ in xrange(10))` 中 gen 是一个 `generator`, 在 `generator` 中变量有自己的一套作用域，与其余作用域空间相互隔离。因此，将会出现这样的 `NameError: name ‘x’ is not defined` 的问题，那么解决方案是什么呢？方案有很多:

In [5]:
class A(object):
    x = 1
    gen = (lambda x: (x for _ in range(10)))(x)
    
print(list(A.gen))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


或者答案可以是这样

In [7]:
class A(object):
    x = 1
    gen = (A.x for _ in range(10))

print(list(A.gen))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## 补充说明

**类(class)中的变量定义的作用域被限制在该类的代码块中，并不会扩展到其方法(methods)的代码块里**，包括其列表生成式(comprehensions) 以及生成器表达式(generator)，因为它们也是用function实现的，那意味着以下代码是错误的：

In [14]:
class A:
    a = 42
    b = list((a + i for i in range(10)))

NameError: name 'a' is not defined

# 2. 最难的装饰器

## 描述

我想写一个**类装饰器**用来度量**函数/方法**运行时间

In [36]:
import time


class Timeit(object):
    def __init__(self, func):
        self._warpped = func
        
    def __call__(self, *args, **kwargs):
        start = time.time()
        result = self._warpped(*args, **kwargs)
        print('cost time: %s' %(time.time() - start))
        return result

**此类装饰器能够运行在普通函数上：**

In [73]:
@Timeit
def func():
    time.sleep(1)
    return 'invoking function func'

print(func())

cost time: 1.0001943111419678
invoking function func


**此类装饰器运行在方法上会报错，为什么?**

In [74]:
class A(object):
    @Timeit
    def func(self):
        time.sleep(1)
        return 'invoking function func'

a = A()
print(a.func())

TypeError: func() missing 1 required positional argument: 'self'

## 答案


使用类装饰器后，在调用 `func` 函数的过程中其对应的 `instance` 并不会传递给 `__call__` 方法，造成其 `mehtod unbound` ,那么解决方法是什么呢？

**方法一：**

描述符


In [43]:
class Timeit(object):
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print('invoking Timer')

    def __get__(self, instance, owner):
        return lambda *args, **kwargs: self.func(instance, *args, **kwargs)


class A(object):
    @Timeit
    def func(self):
        time.sleep(1)
        return 'invoking function func'


a = A()
print(a.func())

invoking function func


**方法二：**

In [42]:
class Foo(object):
    def __init__(self):
        pass

    def __call__(self, func):
        def _call(*args,  **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            print('cost time: %s' %(time.time() - start))
            return result

        return _call


class Bar(object):
    @Foo()
    def bar(self, test, ids):   # bar = Foo()(bar)
        time.sleep(1)
        print('bar')


Bar().bar('aa', 'ids')

bar
cost time: 1.0006234645843506


# 3. Python 调用机制

## 描述

Python开发者们知道 `__call__` 方法可以用来重载圆括号调用，好的，以为问题就这么简单？Naive！

In [80]:
class A(object):
    def __call__(self):
        print("invoking __call__ from A !")

a = A()
a()

invoking __call__ from A !


现在大家可以看到 a() 似乎等价于 `a.__call__() `,好的，现在我们写出了如下的代码

In [88]:
a.__call__ = lambda : "invoking __call__ from lambda !"
a.__call__()

'invoking __call__ from lambda !'

In [89]:
a()

invoking __call__ from A !


**为什么`a()` 没有调用 `a.__call__()` ？** (此题由 USTC 王子博前辈提出)

## 答案

原因在于，在Python中，**新式类(new class) 的内建特殊方法，和实例的属性字典是相互隔离的**，具体可以看看Python的官方文档对于这一情况的说明。

>For new-style classes, implicit invocations of special methods are only guaranteed to work correctly if defined on an object’s type, not in the object’s instance dictionary. That behaviour is the reason why the following code raises an exception (unlike the equivalent example with old-style classes):

同时官方也给出了一个例子：

In [78]:
class C(object):
    pass

c = C()
c.__len__ = lambda: 5
len(c)

TypeError: object of type 'C' has no len()

In [79]:
c.__len__()

5

In [92]:
# 属性中有 `__len__`
c.__dict__

{'__len__': <function __main__.<lambda>()>}

In [91]:
# 然而这里没有
type(c).__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'C' objects>,
              '__weakref__': <attribute '__weakref__' of 'C' objects>,
              '__doc__': None})

## 补充说明

回到我们的例子上来，当我们在执行 `a.call=lambda: "invoking call from lambda"` 时，的确在我们在 `a.__dict__` 中新增加了一个 key 为 `__call__` 的 item，但是当我们执行 `a()` 时，因为涉及特殊方法的调用，因此我们的调用过程不会从 `a.__dict__` 中寻找属性，而是从 `type(a).__dict__` 中寻找属性。因此，就会出现如上所述的情况。

# 4.描述符（待证明）

**尚未搞清楚疑问，不必阅读此章节。**

## 描述

我想写一个 Exam 类，其属性 math 为 [0,100] 的整数，若赋值时不在此范围内则抛出异常，我决定用**描述符**来实现这个需求。

In [112]:
class Grade(object):
    def __init__(self):
        self._score = 0

    def __get__(self, instance, owner):
        return self._score

    def __set__(self, instance, value):
        if 0 <= value <= 100:
            self._score = value
        else:
            raise ValueError('grade must be between 0 and 100')


class Exam(object):
    math = Grade()

    def __init__(self, math):
        self.math = math


niche = Exam(math=90)
print(niche.math)

90


In [113]:
snake = Exam(math=75)
print(snake.math)

75


In [114]:
snake.math = 120

ValueError: grade must be between 0 and 100

---
<br>
看起来一切正常。不过这里面有个巨大的问题，尝试说明是什么问题
为了解决这个问题，我改写了 Grade 描述符如下：

In [11]:
class Grade(object):
    def __init__(self):
        self._grade_pool = {}

    def __get__(self, instance, owner):
        print(self._grade_pool )
        return self._grade_pool.get(instance, None)

    def __set__(self, instance, value):
        if 0 <= value <= 100:
            _grade_pool = self.__dict__.setdefault('_grade_pool', {})
            _grade_pool[instance] = value
        else:
            raise ValueError("fuck")


class Exam(object):
    math = Grade()

    def __init__(self, math):
        self.math = math


# niche = Exam(math=90)
# print(niche.math)

# snake = Exam(math=75)
# print(snake.math)

# print(niche.math)

In [12]:
niche = Exam(math=90)

In [13]:
print(niche.math)

{<__main__.Exam object at 0x0000018E7845B278>: 90}
90


In [14]:
snake = Exam(math=75)
print(snake.math)

{<__main__.Exam object at 0x0000018E7845B278>: 90, <__main__.Exam object at 0x0000018E78454EB8>: 75}
75


In [15]:
print(niche.math)

{<__main__.Exam object at 0x0000018E7845B278>: 90, <__main__.Exam object at 0x0000018E78454EB8>: 75}
90


不过这样会导致更大的问题，请问该怎么解决这个问题？

## 答案

1.第一个问题的其实很简单，如果你再运行一次 `print(niche.math)` 你就会发现，输出值是 `75 `，那么这是为什么呢？这就要先从 Python 的调用机制说起了。我们**如果调用一个属性，那么其顺序是优先从实例的` __dict__` 里查找，然后如果没有查找到的话，那么依次查询类字典，父类字典，直到彻底查不到为止**。好的，现在回到我们的问题，我们发现，在我们的类 `Exam` 中，其 `self.math` 的调用过程是，首先在实例化后的实例的 `__dict__` 中进行查找，没有找到，接着往上一级，在我们的类 `Exam` 中进行查找，好的找到了，返回。那么这意味着，我们对于 `self.math` 的所有操作都是对于类变量` math` 的操作。因此造成变量污染的问题。那么该则怎么解决呢？很多同志可能会说，恩，在 `__set__` 函数中将值设置到具体的实例字典不就行了。

那么这样可不可以呢？答案是，很明显不得行啊，至于为什么，就涉及到我们 Python 描述符的机制了，描述符指的是实现了描述符协议的特殊的类，三个描述符协议指的是 `__get__` , `__set__` , `__delete__`以及 Python 3.6 中新增的 `__set_name__` 方法，其中实现了 `__get__` 以及 `__set__` / `__delete__` / `__set_name__` 的是 `Data descriptors` ，而只实现了 `__get__` 的是 `Non-Data descriptor` 。那么有什么区别呢，前面说了， 我们如果调用一个属性，那么其顺序是优先从实例的 `__dict__` 里查找，然后如果没有查找到的话，那么一次查询类字典，父类字典，直到彻底查不到为止。 **但是，这里没有考虑描述符的因素进去，如果将描述符因素考虑进去，那么正确的表述应该是我们如果调用一个属性，那么其顺序是优先从实例的 `__dict__` 里查找，然后如果没有查找到的话，那么依次查询类字典，父类字典，直到彻底查不到为止。其中如果在类实例字典中的该属性是一个 `Data descriptors` ，那么无论实例字典中存在该属性与否，无条件走描述符协议进行调用，在类实例字典中的该属性是一个 `Non-Data descriptors` ，那么优先调用实例字典中的属性值而不触发描述符协议，如果实例字典中不存在该属性值，那么触发 `Non-Data descriptor` 的描述符协议。回到之前的问题，我们即使在 set 将具体的属性写入实例字典中，但是由于类字典中存在着 `Data descriptors` ，因此，我们在调用 `math` 属性时，依旧会触发描述符协议。

2.经过改良的做法，利用 `dict` 的 key 唯一性，将具体的值与实例进行绑定，但是同时带来了**内存泄露**的问题。那么为什么会造成内存泄露呢，首先复习下我们的 `dict` 的特性，`dict` 最重要的一个特性，就是凡可 hash 的对象皆可为 key ，`dict` 通过利用的 hash 值的唯一性（严格意义上来讲并不是唯一，而是其 hash 值碰撞几率极小，近似认定其唯一）来保证 key 的不重复性，同时（敲黑板，重点来了），**dict 中的 key 引用是强引用类型，会造成对应对象的引用计数的增加，可能造成对象无法被 gc ，从而产生内存泄露。**那么这里该怎么解决呢？两种方法

第一种：

In [ ]:
class Grad(object):
    def __init__(self):
        import weakref
        self._grade_pool = weakref.WeakKeyDictionary()

    def __get__(self, instance, owner):
        return self._grade_pool.get(instance, None)

    def __set__(self, instance, value):
        if 0 <= value <= 100:
            _grade_pool = self.__dict__.setdefault('_grade_pool', {})
            _grade_pool[instance] = value
        else:
            raise ValueError("fuck")

weakref 库中的 `WeakKeyDictionary` 所产生的字典的 key 对于对象的引用是弱引用类型，其不会造成内存引用计数的增加，因此不会造成内存泄露。同理，如果我们为了避免 value 对于对象的强引用，我们可以使用 `WeakValueDictionary` 。

<br>
第二种：

在 Python 3.6 中，实现的 PEP 487 提案，为描述符新增加了一个协议，我们可以用其来绑定对应的对象：

<br>

In [ ]:
class Grad(object):
    def __get__(self, instance, owner):
        return instance.__dict__[self.key]

    def __set__(self, instance, value):
        if 0 <= value <= 100:
            instance.__dict__[self.key] = value
        else:
            raise ValueError("fuck")

    def __set_name__(self, owner, name):
        self.key = name

这道题涉及的东西比较多，这里给出一点参考链接，invoking-descriptors , Descriptor HowTo Guide , PEP 487 , what`s new in Python 3.6 。

<br>

# 5. Python 继承机制

## 描述

试求出以下代码的输出结果。

In [50]:
import time


class Init(object):
    def __init__(self, value):
        self.val = value
        print('init', self.val)
        
class Add2(Init):
    def __init__(self, val):
        super(Add2, self).__init__(val)
        self.val += 2
        print('add',self.val)
        
class Mul5(Init):
    def __init__(self, val):
        super(Mul5, self).__init__(val)
        self.val *= 5
        print('mult', self.val)
        
class Pro(Mul5, Add2):
    pass

class Incr(Pro):
    csup = super(Pro)
    def __init__(self, val):
        self.csup.__init__(val)
        self.val += 1
        
p = Incr(5)

init 5
add 7
mult 35


In [47]:
print(p.val)
# (5 + 2) * 5 + 1

36


## 答案

输出是 36 ，具体可以参考 New-style Classes , multiple-inheritance

# 6. Python 特殊方法

疑问<https://www.oschina.net/question/3306142_2245351?sort=time>



# 可变类型参数

可变参数的默认值，为避免出错应设为`None`

In [23]:
def foo(mydict={}): 
    mydict['name'] = 12
    return mydict

In [25]:
foo()

{'name': 12}

In [26]:
foo({'age':12})

{'age': 12, 'name': 12}